In [10]:
import numpy as np
import pandas as pd
import string, xlrd, nltk, unicodedata, pickle, os, re, time
from nltk import word_tokenize
from collections import defaultdict, Counter
from difflib import SequenceMatcher as textSimilarity # for measuring similarity of text
import pickle

from bs4 import BeautifulSoup

start = time.time()

def nykredit_xls_to_dataframe():
    """
    Parses Nykredit_feed.xls and returns selected columns of the spreadsheet as a Pandas dataframe
    """
    try:
        NK_data = pickle.load(open("NK_data", "rb"))
        print ("NK_data loaded")
    except:        
        # loads the spreadsheet as a pandas.DataFrame
        xls_file = pd.ExcelFile("Nykredit_feed.xls")
#        print (xls_file.sheet_names)
        nykredit_data = xls_file.parse('tmp62.tmp')
        nykredit_text = xls_file.parse('ArticleText')
        
        # choose which columns to use from the spreadsheet:
        selected_parameters = ['ArticleKey', 'ArticleDate', 'QualitativeScore', 'Headline', 'Kilde', 'Raw Xml']
        NK_data = nykredit_data.reindex(columns=selected_parameters)

        # add a column containing the texts extracted from the xml:
        NK_data['Text'] = NK_data.apply(extract_text_from_xml, axis=1)
        # change column name 'Kilde'=>'Source'
        NK_data.columns = [['ArticleKey', 'ArticleDate', 'QualitativeScore', 'Headline', 'Source', 'Raw Xml', 'Text']]
        pickle.dump(NK_data, open("NK_data", "wb"))
        
    return NK_data

def extract_text_from_xml(row):
    """
    Parses the raw XML string in row and returns the text/article contained in row.XML
    """    
    soup = BeautifulSoup(row['Raw Xml'], 'lxml')
    p_blocks = soup.findAll('p')
    
    output = []
    for p in p_blocks:
        if p.string: # some p blocks contains a None element
            output.append(p.string)
    return " ".join(output)

def tokenize_text(row):
    """
    Returns tokens in row.Text using nltk.word_tokenize
    """
    return word_tokenize(row.Text)

NK_data = nykredit_xls_to_dataframe()

NK_data loaded


In [22]:
def duplicate_indexes(dataFrame):
    """
    Returns the indexes of duplicate and semi duplicates texts in dataFrame.
    """
    try:
        duplicates_list = pickle.load(open("duplicates", "rb"))
        print ("duplicates loaded")
    except:
        duplicates_list = []
        i = 0
        while i<dataFrame.shape[0]-1:
            if not i in duplicates_list: # text[i] is a duplicate => no need to check
                j = i+1
                while j<dataFrame.shape[0]:
                    if not j in duplicates_list: # text[j] is a duplicate => no need to check
                        text_similarity = textSimilarity(None, dataFrame.ix[i].Text, dataFrame.ix[j].Text).quick_ratio()
                        if text_similarity > 0.95:
                            duplicates_list.append(j)
                            print((i, j))
                            j += 1
                    j += 1
            i += 1
        pickle.dump(duplicates_list, open("duplicates", "wb"))
    return duplicates_list

duplicates = duplicate_indexes(NK_data)
NK_unique_data = pd.DataFrame.copy(NK_data.drop(duplicates))

duplicates loaded


In [12]:
NK_unique_data.QualitativeScore.value_counts()

 1    714
 0    642
-1     37
Name: QualitativeScore, dtype: int64

In [13]:
neutral_index = np.arange(NK_unique_data.ix[(NK_unique_data.QualitativeScore==-1)].shape[0])
NK_neutral = NK_unique_data.ix[(NK_unique_data.QualitativeScore==-1)]
NK_neutral

,ArticleKey,ArticleDate,QualitativeScore,Headline,Kilde,Raw Xml,Text
29,e240d5a0,2010-10-20,-1,Realkreditten tager sig godt betalt,DR1 Magasinet Penge kl. 21.25 (resumé),"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",\n R...
35,e2454c09,2010-11-02,-1,"Risikovillig realkredit-kapital, tak",Fyns Amts Avis,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",Fra 1991 begyndte jeg at arbejde på Langeland ...
127,e2467b44,2010-11-04,-1,Forbrugerråd: Ejendomsmæglere omgår loven,Sn.dk (Sjællands Nyheder),"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...","Ejendomsmæglere omgår loven, når de hjælper sæ..."
131,e2467b92,2010-11-04,-1,Forbrugerråd: Ejendomsmæglere omgår loven,Lemvig-Folkeblad.dk,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...","Ejendomsmæglere omgår loven, når de hjælper sæ..."
142,e2467c9e,2010-11-04,-1,Forbrugerråd: Ejendomsmæglere omgår loven,SkiveFolkeblad.dk,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...","Ejendomsmæglere omgår loven, når de hjælper sæ..."
171,e2467ba5,2010-11-04,-1,Forbrugerråd: Ejendomsmæglere omgår loven,Børsen.dk,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",Når en sælger skal have udarbejdet tilstandsra...
202,e24637bb,2010-11-03,-1,Tilstandsrapporter lokker huskøbere i en fælde,DR.dk,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...","Reglerne foreskriver godt nok, at det er sælge..."
221,e24704ea,2010-11-05,-1,Advokater vil arbejde for ændrede regler for h...,Skive Folkeblad,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",KØBENHAVN: Mange huskøbere læser dybt i tilsta...
242,e2469601,2010-11-04,-1,Advokater vil ændre regler for huseftersyn,Dagbladet Køge/Ringsted/Roskilde,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",DANMARK: Mange huskøbere læser dybt i tilstand...
243,e2469609,2010-11-04,-1,Mæglere omgår loven,Dagbladet Køge/Ringsted/Roskilde,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",DANMARK: Når en sælger skal have udarbejdet ti...


In [15]:
textSimilarity(None, NK_neutral.ix[127].Text, NK_neutral.ix[221].Text).quick_ratio()

0.8573446327683616

In [16]:
counter = 0
fq = nltk.FreqDist()
while counter<10:
    for index, row in NK_unique_data.iterrows():
        fq += nltk.FreqDist(word_tokenize(row.Text))
        counter += 1
fq

Counter({'ugeavisen': 5,
         'adgangen': 4,
         'fravær': 4,
         'børnetøjsforretning': 1,
         'forberedelse': 5,
         'avlscentre': 1,
         'mark': 11,
         'småpenge': 2,
         'fru': 1,
         'varm': 3,
         'Overblik': 1,
         'Desuden': 54,
         'forsknings-': 3,
         'ophæves': 1,
         'Lånepakken': 4,
         'metode': 8,
         'bæredygtig': 5,
         'brolægger': 1,
         'timen': 1,
         '103,1': 1,
         '184.000': 1,
         'daginstitutioner': 1,
         'energipriser': 7,
         'entreprenørselskab': 1,
         'jordbunden': 1,
         'kirurgiske': 1,
         'gældsproblemerne': 3,
         'arbejdsløs': 3,
         'indbudt': 1,
         'renteforhøjelser': 12,
         'smertehelvede': 1,
         '»Udfordrende': 1,
         '45': 30,
         'Kunsthåndværk': 1,
         'indkøbscentret': 1,
         'gorrissen': 2,
         'Winther': 6,
         'Eksport': 3,
         'stated': 1,
      

In [20]:
fq.most_common(5)

[(',', 34079), ('.', 28991), ('i', 15837), ('at', 12984), ('er', 11606)]